Read Water quality parameters from shapefile
List all satellite dates 
find nearest satellite image for each measurement
associate satellite bands for each measurement
train models 
evaluate results 


In [ ]:
 
#Uncomment the satellite to run
satellite="Landsat" 
satellite='Sentinel_2'
satellite='Planet_Scope'

In [ ]:
#mport xarray as xr
import numpy as np
import pandas as pd
import os
import geopandas as gpd
#initialize variables 
DATA_DIR='DATA/vrana/Landsat_new'
file_bands='DATA/bands_landsat.csv'
band_names = ["B01","B02","B03","B04","B05","B06","B07", "B10"]


#define correct variables for each satellite 
if satellite== 'Landsat':
    DATA_DIR='DATA/vrana/Landsat_new'
    file_bands='DATA/bands_landsat.csv'

 
    band_names = ["B01","B02","B03","B04","B05","B06","B07", "B10"]
if satellite=='Sentinel_2':
    DATA_DIR='DATA/vrana/Sentinel_new'
    file_bands='DATA/bands_sentinel.csv'

    band_names = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']
  
if satellite=='Planet_Scope':
    DATA_DIR='DATA/vrana/transfer.pcloud.PnXOdLhM'
    band_names = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8' ]
    file_bands='DATA/bands_planet.csv'

 
print(satellite)
in_sutu_m = gpd.read_file('DATA/vrana/in_situ_230_standardize_32633.shp')


In [ ]:
import os
import re
from datetime import datetime
import pandas as pd
import geopandas as gpd
import rasterio

base_path = DATA_DIR
 
if os.path.exists(file_bands):
    print("File exists!")
    df=pd.read_csv(file_bands)
else:
    in_sutu_m['date']=pd.to_datetime(in_sutu_m['DATUM'])
    dates = []
    folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]

    for folder in folders:
        # Match the date pattern YYYYMMDD
        match = re.search(r"(\d{8})", folder)
        if match:
            date_str = match.group(1)  # '20230716'
            date_obj = datetime.strptime(date_str, "%Y%m%d")
            dates.append(date_obj)

    print("Extracted dates:", dates)
    print("File does not exist.") 
    # --- Step 1: Extract folder dates and map to folder paths ---
    folder_date_map = {}
    for f in os.listdir(base_path):
        full_path = os.path.join(base_path, f)
        if os.path.isdir(full_path):
            match = re.search(r"(\d{8})", f)
            if match:
                date_obj = datetime.strptime(match.group(1), "%Y%m%d")
                folder_date_map[date_obj] = full_path

    folder_dates = sorted(folder_date_map.keys())

    # --- Step 2: Example gdf with geometry + date column ---
    # gdf = gpd.read_file("points.geojson")
    # gdf['date'] = pd.to_datetime(gdf['date'])
    gdf=in_sutu_m.copy()
    gdf['datum_Sat']=gdf['date']
    # --- Helper to find nearest available folder date ---
    def find_nearest_date(target_date, date_list):
        return min(date_list, key=lambda d: abs(d - target_date))

    # --- Step 3: Extract pixel values into separate band columns ---
    
    # Prepare empty columns in gdf
    for b in band_names:
        gdf[b] = None

    for idx, row in gdf.iterrows():
        nearest_date = find_nearest_date(row['date'], folder_dates)
        folder_path = folder_date_map[nearest_date]
        print(row['date'],nearest_date,folder_path)

        point = row.geometry

        # Read TIFFs from folder
        tiff_files = sorted([f for f in os.listdir(folder_path) if (f.lower().endswith('.tiff') or f.lower().endswith('.tif'))])

        for tiff_file in tiff_files:
            # Extract band code from filename (e.g., 'B04')
            if satellite=="Planet_Scope":
                    match_band = re.search(r'B(\d{1,2}[A]?)', tiff_file)
            else:
                    match_band = re.search(r'_B(\d+[A]?)_', tiff_file)
            if match_band:
                band_code = "B" + match_band.group(1)

                if band_code in band_names:
                    with rasterio.open(os.path.join(folder_path, tiff_file)) as src:
                        # Ensure CRS matches
                        if gdf.crs != src.crs:
                            point = gdf.to_crs(src.crs).iloc[idx].geometry
                        val = list(src.sample([(point.x, point.y)]))[0][0]
                        gdf.at[idx, band_code] = val
                        gdf.at[idx, 'datum_Sat'] = nearest_date
                    
                        print(idx,row['date'],band_code,val,nearest_date )
    df=gdf.dropna().copy()
    df.to_csv(file_bands)


In [ ]:
df

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Regression models
 
from sklearn.linear_model import  RANSACRegressor, PoissonRegressor, LinearRegression, Ridge

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

# ----- CONFIGURATION -----
# Define input and output columns
#in_X_cols = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']
#out_Y_cols = ['IRB_EC_MIN', 'IRB_TUR_MI', 'IRB_WT_MIN', 'IRB_DO_MIN']
out_Y_cols =['IRB_EC', 'IRB_TUR', 'IRB_WT', 'IRB_DO']

in_X_cols=band_names
# ----- LOAD & PREPARE DATA -----
 # Replace with your actual DataFrame
df = df.dropna(subset=in_X_cols + out_Y_cols)

X = df[in_X_cols].values
y = df[out_Y_cols].values  # multivariate regression

# Standardize features
'''scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)'''

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




# ----- MODELS TO BENCHMARK -----
# ----- MODELS TO BENCHMARK -----
models1 = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, n_jobs=-1),
    "Support Vector Regressor": SVR(),
    'RANSACRegressor': RANSACRegressor(),
     "KNN Regressor": KNeighborsRegressor(),
    'PoissonRegressor': PoissonRegressor()
}

# ----- BENCHMARK LOOP -----
print(f"{'Model':<25} {'MAE':>10} {'RMSE':>10} {'R²':>10}")
print("-" * 60)

for name, model in models1.items():
    try:
        # For models that do not support multi-output, loop over targets
        
        if y_train.shape[1] > 1 and not hasattr(model, 'multioutput'):
            preds = []
            for i in range(y_train.shape[1]):
                model.fit(X_train, y_train[:, i])
                pred = model.predict(X_test)
                preds.append(pred)
            y_pred = np.stack(preds, axis=1)
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

        # Inverse scale
        #y_test_orig = scaler_y.inverse_transform(y_test)
        #y_pred_orig = scaler_y.inverse_transform(y_pred)

        # Compute metrics averaged across all output variables
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        print(f"{name:<25} {mae:10.3f} {rmse:10.3f} {r2:10.3f}")
    except Exception as e:
        print(f"{name:<25} ERROR: {e}")


In [ ]:
from sklearn.linear_model import BayesianRidge, RANSACRegressor, PoissonRegressor, LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
X = df[in_X_cols].values
y = df[out_Y_cols].values  # multivariate regression

# Standardize features
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ----- MODELS TO BENCHMARK -----
models1 = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
        "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, n_jobs=-1),

    "Support Vector Regressor": SVR(),
    'RANSACRegressor': RANSACRegressor(),
     "KNN Regressor": KNeighborsRegressor(),
    'PoissonRegressor': PoissonRegressor()
}
n_targets = y_train.shape[1]

# ----- LATEX HEADER -----
print("\\begin{table}[h]")
print("\\centering")

# column format: 1 for model + 1 per target
col_format = "l" + "c" * n_targets
print(f"\\begin{{tabular}}{{{col_format}}}")
print("\\hline")

# header row
header = "Model"
for i in range(n_targets):
    header += f" & {out_Y_cols[i]} (MAE, RMSE, R\\^2)"
print(header + " \\\\")
print("\\hline")

# ----- BENCHMARK LOOP -----
for name, model in models1.items():
    row = name
    try:
        for i in range(n_targets):
            # Train on single target
            model.fit(X_train, y_train[:, i])
            y_pred = model.predict(X_test)

            # Construct placeholders for scaler (full shape)
            y_test_full = np.zeros_like(y_test)
            y_pred_full = np.zeros_like(y_test)
            y_test_full[:, i] = y_test[:, i]
            y_pred_full[:, i] = y_pred

            # Inverse scaling
            y_test_orig = scaler_y.inverse_transform(y_test_full)[:, i]
            y_pred_orig = scaler_y.inverse_transform(y_pred_full)[:, i]

            # Metrics
            mae = mean_absolute_error(y_test_orig, y_pred_orig)
            rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
            r2 = r2_score(y_test_orig, y_pred_orig)

            row += f" & ({mae:.3f}, {rmse:.3f}, {r2:.3f})"
    except Exception as e:
        row += " & ERROR" * n_targets

    print(row + " \\\\")

# ----- LATEX FOOTER -----
print("\\hline")
print("\\end{tabular}")
print("\\caption{Benchmark results of regression models for each target variabl for "+ satellite +"}")
print("\\label{tab:benchmark_results}")
print("\\end{table}")
